# PSF and transformation estimation tutorial

* Estimate a point spread function with subpixel resolution from a sparsely filled image
* Estimate the affine transformation between site and image coordinates
* **Make sure you have run the tutorial `image_generation.ipynb`**

In [ ]:
import state_reconstruction as srec

import numpy as np
import os

from libics.env import DIR_DESKTOP
from libics.core import io
from libics.tools import plot
from libics.tools.trafo.linear import AffineTrafo2d

## Package configuration

**The `state_reconstruction` package configuration**

* can be read as follows
* and can be altered by overwriting the appropriate parameters in the configuration file located at `~/.libics/state_estimation/config.json`, where `~` indicates the user folder (e.g. `C:/Users/<my_user_name>`

In [ ]:
srec.get_config()

**Files generated using this tutorial**

* are saved to the desktop with the following file name
* and may be used by other tutorials

In [ ]:
DEMO_FILENAME = "srec_demo"

## Load prerequisites

**Load affine transformation object**

In [ ]:
filepath_trafo_site_to_image = os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_trafo.json")
trafo_site_to_image = io.load(filepath_trafo_site_to_image)
trafo_site_to_image

In [ ]:
phase_ref_image = srec.get_config("trafo_gen.phase_ref_image")
phase_ref_site = srec.get_config("trafo_gen.phase_ref_site")
phase_ref_image, phase_ref_site

**Load PSF object**

In [ ]:
filepath_ipsf_gen = os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_psf.json")
ipsf_gen = srec.IntegratedPsfGenerator.load(filepath_ipsf_gen)
ipsf_gen

**Load image to be reconstructed**

In [ ]:
demo_image = io.load(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_image-sparse.png")
)
demo_occ = io.load(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_occ-sparse.json")
)

## Supersampling point spread function estimator

**Initialization with guessed PSF**

* The PSF estimator object requires PSF initialization
* We use a guessed Gaussian PSF

In [ ]:
psf_supersample = 5
psf_size = 21

guess_psf_width = srec.get_psf_gaussian_width(
    780e-9, px_size=532e-9/4.2, numerical_aperture=0.63
)
guess_psf_integrated = srec.get_psf_gaussian(
    wx=guess_psf_width, size=psf_size
)
plot.pcolorim(guess_psf_integrated)

**Set up PSF estimator object**

* Initialize estimator with guessed PSF
* Set parameters to identify isolated atoms
* The filtering and identification parameters strongly depend on the image quality; especially for real-life images, playing with these parameters is typically required
* Finally, the supersampling parameters (for estimating subpixel resolution) can be set
* Note that it might be advisable to crop the raw images suitably first

In [ ]:
psf_est = srec.SupersamplePsfEstimator(
    # Guessed PSF
    psf_integrated=guess_psf_integrated,
    # Spatial filtering parameters
    filter_inner_thr=1/np.e,
    filter_outer_thr=1/np.e**4,
    # Histogram thresholding parameters
    onsite_split_cond_width=6,
    neighbor_split_cond_width=9,
    # Local cropping parameters
    label_center_rel_im_size=2,
    # Supersampling parameters
    psf_supersample=psf_supersample,
    psf_interpolation="linear"
)
psf_est.setup()
psf_est

In [ ]:
# Check whether isolated atoms were detected well
_lcs = psf_est.get_label_centers(demo_image)
plot.figure(figsize=(14, 12))
plot.pcolorim(demo_image, cmap="hot", vmin=100, vmax=1000, colorbar=True)
plot.scatter(*_lcs.T, marker=".", color="blue")

**Estimate supersampled PSF**

* Use images with sparse atoms to estimate PSF
* Note that you can use more than a single image (as in this example)

In [ ]:
# Get all sparse atom images (here only one)
multiple_images = [demo_image]

In [ ]:
# Estimate PSF
estimated_psf = psf_est.get_psf(*multiple_images, print_progress=True)
# Zero and normalize estimated PSF
estimated_psf -= np.percentile(estimated_psf, 20)
estimated_psf[estimated_psf < 0] = 0
estimated_psf /= np.sum(estimated_psf)

In [ ]:
# Check
fig, axs = plot.subplots(figsize=(16, 2.5), ncols=5)

# Linear color scale
plt_params = dict(
    colorbar=True, cmap="viridis", vmin=0, vmax=None, clabel="linear"
)
plot.pcolorim(estimated_psf, ax=axs[0], title="estimated", **plt_params)
plot.pcolorim(ipsf_gen.psf, ax=axs[1], title="real", **plt_params)

# Linear difference
plt_params = dict(
    colorbar=True, cmap="RdBu_r", vcen=0, vdif=True, clabel="linear"
)
plot.pcolorim(estimated_psf - ipsf_gen.psf, ax=axs[2], title="difference", **plt_params)

# Log color scale
vmin = -2
plt_params = dict(
    colorbar=True, cmap="viridis", vmin=vmin, vmax=0, clabel="log10"
)
estimated_psf_log = np.log10(estimated_psf/estimated_psf.max())
estimated_psf_log[estimated_psf_log < vmin] = vmin
real_psf_log = np.log10(ipsf_gen.psf/ipsf_gen.psf.max())
real_psf_log[real_psf_log < vmin] = vmin
plot.pcolorim(estimated_psf_log, ax=axs[3], title="estimated", **plt_params)
plot.pcolorim(real_psf_log, ax=axs[4], title="real", **plt_params)

plot.style_figure(tight_layout=True)

In [ ]:
# Save estimated PSF as PSF generator object for further use
estimated_ipsf_gen = srec.IntegratedPsfGenerator(
    psf=estimated_psf,
    psf_supersample=psf_supersample
)
estimated_ipsf_gen.save(os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_estimated-psf"))

## Afine transformation estimator

**Initialization with guessed trafo**

* The transformation estimator object requires a guessed initialization

In [ ]:
# We purposefully set slightly wrong transformation parameters for initialization
guess_trafo = AffineTrafo2d()
guess_trafo.set_origin_axes(
    magnification=[4, 4],
    angle=np.deg2rad([43, 43])
)
guess_trafo

**Set up isolated atom locator object**

* This object is a parent class of `SupersamplePsfEstimator`
* and uses the same parameters to identify isolated atoms (see above)

In [ ]:
iso_loc = srec.IsolatedLocator(
    # Guessed PSF
    psf_integrated=guess_psf_integrated,
    # Spatial filtering parameters
    filter_inner_thr=1/np.e,
    filter_outer_thr=1/np.e**4,
    # Histogram thresholding parameters
    onsite_split_cond_width=6,
    neighbor_split_cond_width=9,
    # Local cropping parameters
    label_center_rel_im_size=2,
)
iso_loc.setup()
iso_loc

In [ ]:
# Check whether isolated atoms were detected well
_lcs = iso_loc.get_label_centers(demo_image)
plot.figure(figsize=(14, 12))
plot.pcolorim(demo_image, cmap="hot", vmin=100, vmax=1000, colorbar=True)
plot.scatter(*_lcs.T, marker=".", color="blue")

**Set up transformation estimator object**

* Using the isolated atom locator, the trafo estimator object can fit an affine transformation
* Note that multiple images can be used for fitting (here only one image is demonstrated)

In [ ]:
trafo_est = srec.TrafoEstimator(
    isolated_locator=iso_loc,
    guess_trafo=guess_trafo,
    angle_range=np.deg2rad(5),
    angle_num=32,
)
trafo_est

In [ ]:
# Set up multiple images for fitting
multiple_images = [demo_image]
# Fit transformation
estimated_trafo = trafo_est.find_trafo(*multiple_images, print_progress=True)
# Set desired offset
estimated_trafo.set_offset_by_point_pair(
    phase_ref_site, phase_ref_image
)
estimated_trafo

In [ ]:
# Compare to real transformation
trafo_site_to_image